In [1]:
import cv2
import face_recognition
import numpy as np
import os
import Arm_Lib
import threading
import ipywidgets as widgets
from IPython.display import display
from armctrl import arm_ctrl

In [2]:
model = "General"

button_layout  = widgets.Layout(width='200px', height='70px', align_self='center')
output = widgets.Output()

exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
retrieve_button = widgets.Button(description='Retrieve', button_style='info', layout=button_layout)
place_button = widgets.Button(description='Place', button_style='info', layout=button_layout)

imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([imgbox, exit_button, retrieve_button, place_button], layout=widgets.Layout(align_self='center'))

In [3]:
def exit_button_Callback(value):
    global model
    model = 'Exit'
    with output: print(model)

def retrieve_button_callback(value):
    global model
    model = "Retrieve"
    with output: print(model)

def place_button_callback(value):
    global model
    model = "Place"
    with output: print(model)

exit_button.on_click(exit_button_Callback)
retrieve_button.on_click(retrieve_button_callback)
place_button.on_click(place_button_callback)

In [35]:
def arm_action(action, place_id):
    arm = arm_ctrl()
    if action == "Retrieve":
        arm.arm_retrieve(place_id)
    elif action == "Place":
        arm.arm_place(place_id)
    else:
        pass  
    arm.move_to_face_capture()          

In [31]:
# Initialize places (False indicates an empty place)
places = [False] * 4  # Simulating 5 available places

# Load or register faces
known_faces_dir = 'known_faces'
if not os.path.exists(known_faces_dir):
    os.makedirs(known_faces_dir)
known_face_encodings = []
known_face_ids = []

# Load known faces
for filename in os.listdir(known_faces_dir):
    face_id = filename.split('.')[0]
    face_id = filename.split('_')[-1]
    face_image = face_recognition.load_image_file(f'{known_faces_dir}/{filename}')
    face_encoding = face_recognition.face_encodings(face_image)[0]
    known_face_encodings.append(face_encoding)
    known_face_ids.append(face_id)

In [37]:
def process_frame(frame, known_face_encodings, known_face_ids, places, known_faces_dir,action):
    print("Starting")
    if frame is not None:
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)
        if face_encodings:
            face_encoding = face_encodings[0]
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            if True in matches:
                first_match_index = matches.index(True)
                face_id = known_face_ids[first_match_index].split('_')[-1]
                face_id = face_id.split('.')[0]
                print(face_id)
                face_id = known_face_ids[int(face_id)-1]
                print(places)
                print(face_id)
                place_id = places.index(face_id)
                print(place_id)
                print(f"Welcome back, person {face_id}.")
                arm_action(action, place_id + 1)
                # Remove face data when the action is 'retrieve'
                if action == 'Retrieve':
                    # Remove face encoding and ID from lists
                    del known_face_encodings[first_match_index]
                    del known_face_ids[first_match_index]
                    # Optionally, delete image file if no longer needed
                    image_path = f'{known_faces_dir}/{face_id}.jpg'
                    if os.path.exists(image_path):
                        os.remove(image_path)
                    
                # Mark the place as empty
                places[place_id] = False  
            else:
                # Register and assign a new place if available
                if False in places:
                    new_place_id = places.index(False)
                    new_face_id = f"person_{len(known_face_encodings) + 1}"
                    places[new_place_id] = new_face_id
                    known_face_encodings.append(face_encoding)
                    known_face_ids.append(new_face_id)
                    cv2.imwrite(f'{known_faces_dir}/{new_face_id}.jpg', frame)
                    print(f"New person registered as {new_face_id}.")
                    arm_action(action, new_place_id + 1)
                else:
                    print("No available places.")
        else:
            print("No face found in the captured image.")
    else:
        print("No frames captured")

In [42]:
# Load known faces
for filename in os.listdir(known_faces_dir):
    face_id = filename.split('.')[0]
    face_id = face_id.split('_')[-1]  # Extract just the numeric part of the file name
    face_image = face_recognition.load_image_file(f'{known_faces_dir}/{filename}')
    face_encoding = face_recognition.face_encodings(face_image)[0]
    known_face_encodings.append(face_encoding)
    known_face_ids.append(face_id)

def process_frame(frame, known_face_encodings, known_face_ids, places, known_faces_dir, action):
    print("Starting")
    if frame is not None:
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)
        if face_encodings:
            face_encoding = face_encodings[0]
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            if True in matches:
                first_match_index = matches.index(True)
                face_id = known_face_ids[first_match_index]
                place_id = int(face_id.split('_')[-1]) - 1  # Adjust for 0-based indexing
                print(f"Welcome back, person {face_id}.")
                arm_action(action, place_id + 1)  # Assuming arm_action is defined elsewhere
                if action == 'Retrieve':
                    # Clear the slot and remove face data
                    places[place_id] = False
                    del known_face_encodings[first_match_index]
                    del known_face_ids[first_match_index]
                    image_path = f'{known_faces_dir}/{face_id}.jpg'
                    if os.path.exists(image_path):
                        os.remove(image_path)
            else:
                if action == 'Place':
                    # Find an empty slot and place the object
                    empty_slot_index = places.index(False)
                    new_face_id = f"person_{len(known_face_encodings) + 1}"
                    places[empty_slot_index] = new_face_id
                    known_face_encodings.append(face_encoding)
                    known_face_ids.append(new_face_id)
                    cv2.imwrite(f'{known_faces_dir}/{new_face_id}.jpg', frame)
                    print(f"New person registered as {new_face_id}.")
                    arm_action(action, empty_slot_index + 1)  # Place object in the empty slot
                else:
                    print("No match found.")
        else:
            print("No face found in the captured image.")
    else:
        print("No frames captured.")

In [43]:
def camera():
    global known_face_encodings, known_face_ids, places, known_faces_dir, model
    capture = cv2.VideoCapture(0)
    while capture.isOpened():
        try:
            ret, frame = capture.read()
            frame = cv2.resize(frame, (640, 480))
            
            if model == 'Place':
                print("Placing")
                process_frame(frame, known_face_encodings, known_face_ids, places, known_faces_dir, "Place")
                model = "General"
                
            if model == "Retrieve":
#                 frame = cv2.resize(frame, (640, 480))
                print("Retrieving")
                process_frame(frame, known_face_encodings, known_face_ids, places, known_faces_dir, "Retrieve")
                model = "General"

                
            if model == "Exit":
                cv2.destroyAllWindows()
                capture.release()
                model="General"
                break
            imgbox.value = cv2.imencode('.jpg', frame)[1].tobytes()

        except KeyboardInterrupt:
            capture.release()

In [47]:
display(controls_box,output)
threading.Thread(target=camera,).start()

Output(outputs=({'name': 'stdout', 'text': 'Place\nRetrieve\nRetrieve\nPlace\nExit\nPlace\nRetrieve\nRetrieve\…

Retrieving
Starting
Welcome back, person person_3.
Retrieving


In [46]:
known_face_ids

['person_1', '1', 'person_3', 'person_4']

In [24]:
places

[False, False, False, False]

In [3]:
import Arm_Lib

def ctrl_all_servo(angles, s_time):
    Arm = Arm_Lib.Arm_Device()
    Arm.Arm_serial_servo_write6_array(angles, s_time)

ctrl_all_servo([90, 53, 33, 36, 90, 30],1000) # id
ctrl_all_servo([90, 110, 40, 40,110, 0],1000) ##face
